In [ ]:
import numpy as np
import scanpy as sc

from scib_metrics.benchmark import Benchmarker
import scib

In [ ]:
adata = sc.read('/gpfs/gibbs/pi/zhao/tl688/cpsc_finalproject/genept_data/GenePT/protein_data/reference.h5ad')
query = sc.read('/gpfs/gibbs/pi/zhao/tl688/cpsc_finalproject/genept_data/GenePT/protein_data/query.h5ad')

adata_ref =sc.AnnData(adata.obsm['protein_counts'], obs = adata.obs)
adata_que =sc.AnnData(query.obsm['pro_exp'], obs = query.obs)
adata_combine = sc.concat([adata_ref, adata_que], keys = ['reference', 'query'])

In [ ]:
import pickle
with open('ensem_emb_pro_pbmcseuratv4.pickle', "rb") as fp: 
    GPT_3_5_gene_embeddings = pickle.load(fp)
gene_names= adata_combine.var_names
count_missing = 0
EMBED_DIM = 1536 # embedding dim from GPT-3.5
lookup_embed = np.zeros(shape=(len(gene_names),EMBED_DIM))
for i, gene in enumerate(gene_names):
    if gene in GPT_3_5_gene_embeddings.keys():
        lookup_embed[i,:] = GPT_3_5_gene_embeddings[gene].flatten()
    else:
        count_missing+=1
lookup_embed.shape

In [ ]:
sc.pp.filter_cells(adata_combine, min_genes=1) # This dataset contains proteins with expression as 0.
adata_combine.obsm['X_proPT'] = adata_combine.X / np.sum(adata_combine.X, axis=1)[:,None]  @ lookup_embed #GPT 3.5 wa

In [ ]:
results = scib.metrics.metrics(
    adata,
    adata_int=adata,
    batch_key="dataset_name",
    label_key="celltype.l2",
    embed='X_proPT',
    isolated_labels_asw_=False,
    silhouette_=True,
    hvg_score_=False,
    graph_conn_=True,
    pcr_=True,
    isolated_labels_f1_=False,
    trajectory_=False,
    nmi_=True,  # use the clustering, bias to the best matching
    ari_=True,  # use the clustering, bias to the best matching
    cell_cycle_=False,
    kBET_=True,  # kBET return nan sometimes, need to examine
    ilisi_=True,
    clisi_=True,
)

In [ ]:
results